# 📊 MarketMind - Análise de Dados e Estudos

Este notebook está configurado para análise exploratória dos dados de ações e estudos do modelo de ML.

**Objetivo:** Facilitar estudos, experimentos e análises sem modificar o código principal da aplicação.

## 1. Importações e Configuração

In [ ]:
# Imports principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Imports de ML
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

# Configuração de plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuração do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Bibliotecas importadas com sucesso!")

## 2. Configuração da API

In [ ]:
# Configuração da API BraPI
API_KEY = "nUUZxG2ZdAWuSkBDhPobC2"
BASE_URL = "https://brapi.dev/api"

def buscar_dados_acao(ticker, range='3mo', interval='1d'):
    """
    Busca dados de uma ação via API BraPI
    
    Args:
        ticker: Código da ação (ex: PETR4)
        range: Período (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)
        interval: Intervalo (1d, 5d, 1wk, 1mo, 3mo)
    
    Returns:
        dict com dados da ação e histórico em DataFrame
    """
    try:
        headers = {"Authorization": f"Bearer {API_KEY}"}
        
        # Dados atuais
        response = requests.get(
            f"{BASE_URL}/quote/{ticker}",
            headers=headers,
            timeout=10
        )
        
        if response.status_code != 200:
            return None, f"Erro {response.status_code}: Ticker não encontrado"
        
        data = response.json()
        if 'results' not in data or not data['results']:
            return None, f"Ticker {ticker} não encontrado"
        
        acao_data = data['results'][0]
        
        # Dados históricos
        response_hist = requests.get(
            f"{BASE_URL}/quote/{ticker}?range={range}&interval={interval}",
            headers=headers,
            timeout=10
        )
        
        historico = None
        
        if response_hist.status_code == 200:
            data_hist = response_hist.json()
            
            if 'results' in data_hist and data_hist['results']:
                hist_data = data_hist['results'][0].get('historicalDataPrice', [])
                
                if hist_data:
                    hist_list = []
                    for item in hist_data:
                        try:
                            hist_list.append({
                                'Data': datetime.fromtimestamp(item['date']),
                                'Open': item.get('open', 0),
                                'High': item.get('high', 0),
                                'Low': item.get('low', 0),
                                'Close': item.get('close', 0),
                                'Volume': item.get('volume', 0)
                            })
                        except (KeyError, ValueError):
                            continue
                    
                    if hist_list:
                        historico = pd.DataFrame(hist_list).set_index('Data').sort_index()
        
        resultado = {
            'ticker': acao_data.get('symbol', ticker),
            'nome': acao_data.get('shortName', acao_data.get('longName', 'N/A')),
            'preco_atual': round(acao_data.get('regularMarketPrice', 0), 2),
            'variacao': round(acao_data.get('regularMarketChangePercent', 0), 2),
            'variacao_valor': round(acao_data.get('regularMarketChange', 0), 2),
            'maxima_dia': round(acao_data.get('regularMarketDayHigh', 0), 2),
            'minima_dia': round(acao_data.get('regularMarketDayLow', 0), 2),
            'volume': acao_data.get('regularMarketVolume', 0),
            'abertura': round(acao_data.get('regularMarketOpen', 0), 2),
            'market_cap': acao_data.get('marketCap', 0),
            'historico': historico
        }
        
        return resultado, None
        
    except Exception as e:
        return None, f"Erro: {str(e)}"

print("✅ Funções de API configuradas!")

## 3. Funções de Cálculo de Indicadores Técnicos

In [ ]:
def calcular_rsi(prices, period=14):
    """Calcula o Relative Strength Index"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return (100 - (100 / (1 + rs))).fillna(50)

def calcular_macd(prices, fast=12, slow=26, signal=9):
    """Calcula MACD e linha de sinal"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    macd_histogram = macd - macd_signal
    return macd.fillna(0), macd_signal.fillna(0), macd_histogram.fillna(0)

def calcular_bollinger_bands(prices, window=20, std_dev=2):
    """Calcula Bandas de Bollinger"""
    ma = prices.rolling(window=window).mean()
    std = prices.rolling(window=window).std()
    upper = ma + (std * std_dev)
    lower = ma - (std * std_dev)
    return upper.fillna(prices), ma.fillna(prices), lower.fillna(prices)

def adicionar_indicadores_tecnicos(df):
    """
    Adiciona todos os indicadores técnicos ao DataFrame
    
    Args:
        df: DataFrame com colunas OHLCV
    
    Returns:
        DataFrame com indicadores adicionados
    """
    df = df.copy()
    
    # Médias móveis
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_12'] = df['Close'].ewm(span=12).mean()
    df['EMA_26'] = df['Close'].ewm(span=26).mean()
    
    # RSI
    df['RSI'] = calcular_rsi(df['Close'])
    
    # MACD
    df['MACD'], df['MACD_signal'], df['MACD_histogram'] = calcular_macd(df['Close'])
    
    # Bollinger Bands
    df['BB_upper'], df['BB_middle'], df['BB_lower'] = calcular_bollinger_bands(df['Close'])
    df['BB_width'] = (df['BB_upper'] - df['BB_lower']) / df['Close']
    df['BB_position'] = (df['Close'] - df['BB_lower']) / (df['BB_upper'] - df['BB_lower'])
    
    # Retornos
    df['Return_1d'] = df['Close'].pct_change()
    df['Return_3d'] = df['Close'].pct_change(3)
    df['Return_5d'] = df['Close'].pct_change(5)
    
    # Volatilidade
    df['Volatility_10d'] = df['Return_1d'].rolling(window=10).std()
    df['Volatility_20d'] = df['Return_1d'].rolling(window=20).std()
    
    # Volume
    df['Volume_SMA_10'] = df['Volume'].rolling(window=10).mean()
    df['Volume_ratio'] = df['Volume'] / df['Volume_SMA_10']
    
    # Indicadores de tendência
    df['Price_above_SMA20'] = (df['Close'] > df['SMA_20']).astype(int)
    df['SMA_trend'] = (df['SMA_5'] > df['SMA_20']).astype(int)
    
    # Razões de preço
    df['High_Low_ratio'] = (df['High'] - df['Low']) / df['Close']
    df['Open_Close_ratio'] = (df['Close'] - df['Open']) / df['Open']
    
    return df

print("✅ Funções de indicadores técnicos prontas!")

## 4. Exemplo de Uso: Análise de uma Ação

In [ ]:
# Buscar dados de uma ação (exemplo: PETR4)
ticker = "PETR4"  # Altere para a ação desejada

dados, erro = buscar_dados_acao(ticker, range='6mo', interval='1d')

if erro:
    print(f"❌ Erro: {erro}")
else:
    print(f"✅ Dados carregados para {dados['ticker']} - {dados['nome']}")
    print(f"Preço atual: R$ {dados['preco_atual']:.2f} ({dados['variacao']:+.2f}%)")
    print(f"Volume: {dados['volume']:,}")
    print(f"\nHistórico: {len(dados['historico'])} dias de dados")
    
    # Visualizar primeiras linhas
    df_historico = dados['historico']
    display(df_historico.head())
    display(df_historico.tail())

## 5. Adicionar Indicadores Técnicos

In [ ]:
# Adicionar indicadores ao DataFrame
if dados and dados['historico'] is not None:
    df_completo = adicionar_indicadores_tecnicos(dados['historico'])
    
    print(f"✅ Indicadores técnicos adicionados!")
    print(f"Total de colunas: {len(df_completo.columns)}")
    print(f"\nColunas disponíveis:")
    print(df_completo.columns.tolist())
    
    # Visualizar dados recentes com indicadores
    display(df_completo[['Close', 'RSI', 'MACD', 'BB_width', 'Volume_ratio']].tail(10))

## 6. Visualização: Preço e Volume

In [ ]:
# Gráfico de preço e volume
if dados and dados['historico'] is not None:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # Preço
    ax1.plot(df_completo.index, df_completo['Close'], linewidth=2, label='Close', color='#00d4ff')
    ax1.plot(df_completo.index, df_completo['SMA_20'], linewidth=1.5, label='SMA 20', color='orange', alpha=0.7)
    ax1.plot(df_completo.index, df_completo['SMA_50'], linewidth=1.5, label='SMA 50', color='red', alpha=0.7)
    ax1.set_ylabel('Preço (R$)', fontsize=12)
    ax1.set_title(f'{ticker} - Preço e Volume', fontsize=14, fontweight='bold')
    ax1.legend(loc='best')
    ax1.grid(True, alpha=0.3)
    
    # Volume
    colors = ['green' if row['Close'] >= row['Open'] else 'red' for idx, row in df_completo.iterrows()]
    ax2.bar(df_completo.index, df_completo['Volume'], color=colors, alpha=0.6)
    ax2.plot(df_completo.index, df_completo['Volume_SMA_10'], linewidth=2, label='Volume Médio (10d)', color='blue')
    ax2.set_ylabel('Volume', fontsize=12)
    ax2.set_xlabel('Data', fontsize=12)
    ax2.legend(loc='best')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Visualização: Indicadores Técnicos

In [ ]:
# Gráfico de indicadores técnicos
if dados and dados['historico'] is not None:
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))
    
    # RSI
    ax1.plot(df_completo.index, df_completo['RSI'], linewidth=2, color='purple')
    ax1.axhline(y=70, color='r', linestyle='--', alpha=0.5, label='Sobrecomprado')
    ax1.axhline(y=30, color='g', linestyle='--', alpha=0.5, label='Sobrevendido')
    ax1.set_title('RSI (Relative Strength Index)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('RSI')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # MACD
    ax2.plot(df_completo.index, df_completo['MACD'], linewidth=2, label='MACD', color='blue')
    ax2.plot(df_completo.index, df_completo['MACD_signal'], linewidth=2, label='Signal', color='red')
    ax2.bar(df_completo.index, df_completo['MACD_histogram'], alpha=0.3, label='Histogram')
    ax2.set_title('MACD', fontsize=12, fontweight='bold')
    ax2.set_ylabel('MACD')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Bollinger Bands
    ax3.plot(df_completo.index, df_completo['Close'], linewidth=2, label='Close', color='#00d4ff')
    ax3.plot(df_completo.index, df_completo['BB_upper'], linewidth=1, label='Upper Band', color='red', linestyle='--')
    ax3.plot(df_completo.index, df_completo['BB_middle'], linewidth=1, label='Middle Band', color='orange')
    ax3.plot(df_completo.index, df_completo['BB_lower'], linewidth=1, label='Lower Band', color='green', linestyle='--')
    ax3.fill_between(df_completo.index, df_completo['BB_upper'], df_completo['BB_lower'], alpha=0.1)
    ax3.set_title('Bollinger Bands', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Preço (R$)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Volatilidade
    ax4.plot(df_completo.index, df_completo['Volatility_10d'] * 100, linewidth=2, label='Volatilidade 10d', color='red')
    ax4.plot(df_completo.index, df_completo['Volatility_20d'] * 100, linewidth=2, label='Volatilidade 20d', color='orange')
    ax4.set_title('Volatilidade', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Volatilidade (%)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 8. Análise Estatística

In [ ]:
# Estatísticas descritivas
if dados and dados['historico'] is not None:
    print("📊 ESTATÍSTICAS DESCRITIVAS\n")
    print("="*60)
    
    stats = df_completo[['Close', 'Volume', 'Return_1d', 'RSI', 'Volatility_20d']].describe()
    display(stats)
    
    print("\n📈 RETORNOS")
    print("="*60)
    print(f"Retorno Total: {((df_completo['Close'].iloc[-1] / df_completo['Close'].iloc[0]) - 1) * 100:.2f}%")
    print(f"Retorno Anualizado: {df_completo['Return_1d'].mean() * 252 * 100:.2f}%")
    print(f"Volatilidade Anualizada: {df_completo['Return_1d'].std() * np.sqrt(252) * 100:.2f}%")
    print(f"Sharpe Ratio (aprox): {(df_completo['Return_1d'].mean() / df_completo['Return_1d'].std()) * np.sqrt(252):.2f}")

## 9. Correlação entre Indicadores

In [ ]:
# Matriz de correlação
if dados and dados['historico'] is not None:
    # Selecionar colunas para correlação
    colunas_corr = ['Close', 'Volume', 'RSI', 'MACD', 'BB_width', 
                    'Volatility_20d', 'Return_1d', 'Volume_ratio']
    
    df_corr = df_completo[colunas_corr].dropna()
    
    # Calcular correlação
    corr_matrix = df_corr.corr()
    
    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Matriz de Correlação - Indicadores Técnicos', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 10. Preparação de Dados para ML (Features)

Esta seção prepara os dados no mesmo formato usado pelo modelo de ML da aplicação.

In [ ]:
def preparar_features_ml(df, dias_previsao=5):
    """
    Prepara features para ML (mesmo formato do app.py)
    """
    df = df.copy()
    
    # Adicionar indicadores se ainda não existem
    if 'RSI' not in df.columns:
        df = adicionar_indicadores_tecnicos(df)
    
    # Features selecionadas (mesmo do app.py)
    feature_cols = [
        'Return_1d', 'Return_3d', 'Return_5d',
        'RSI', 'MACD', 'MACD_histogram',
        'BB_width', 'BB_position',
        'Volatility_10d', 'Volatility_20d',
        'Volume_ratio', 'High_Low_ratio', 'Open_Close_ratio',
        'Price_above_SMA20', 'SMA_trend'
    ]
    
    # Remover NaNs
    df_clean = df.dropna()
    
    # Preparar X (features)
    X = df_clean[feature_cols].values
    
    # Preparar y (targets - retornos futuros)
    y = []
    for i in range(len(df_clean) - dias_previsao):
        current_price = df_clean['Close'].iloc[i]
        future_returns = []
        for j in range(1, dias_previsao + 1):
            if i + j < len(df_clean):
                future_price = df_clean['Close'].iloc[i + j]
                ret = (future_price - current_price) / current_price
                future_returns.append(ret)
        
        if len(future_returns) == dias_previsao:
            y.append(future_returns)
    
    X_final = X[:len(y)]
    y_final = np.array(y)
    
    return X_final, y_final, df_clean, feature_cols

# Preparar dados
if dados and dados['historico'] is not None:
    X, y, df_clean, features = preparar_features_ml(df_completo)
    
    print(f"✅ Dados preparados para ML!")
    print(f"Shape de X: {X.shape}")
    print(f"Shape de y: {y.shape}")
    print(f"\nFeatures utilizadas ({len(features)}):")
    for i, feat in enumerate(features, 1):
        print(f"  {i:2d}. {feat}")

## 11. Comparação de Múltiplas Ações

Analise múltiplas ações simultaneamente.

In [ ]:
# Lista de tickers para comparar
tickers_comparar = ['PETR4', 'VALE3', 'ITUB4', 'BBDC4', 'MGLU3']

# Buscar dados de todos
dados_multiplos = {}

for ticker in tickers_comparar:
    dados_acao, erro = buscar_dados_acao(ticker, range='3mo', interval='1d')
    if not erro and dados_acao:
        dados_multiplos[ticker] = dados_acao
        print(f"✅ {ticker}: {dados_acao['nome']}")
    else:
        print(f"❌ {ticker}: {erro}")

# Criar DataFrame comparativo
if dados_multiplos:
    df_comparativo = pd.DataFrame({
        'Ticker': [d['ticker'] for d in dados_multiplos.values()],
        'Nome': [d['nome'] for d in dados_multiplos.values()],
        'Preço': [d['preco_atual'] for d in dados_multiplos.values()],
        'Variação (%)': [d['variacao'] for d in dados_multiplos.values()],
        'Volume': [d['volume'] for d in dados_multiplos.values()],
    })
    
    print("\n📊 COMPARATIVO DE AÇÕES\n")
    display(df_comparativo)
    
    # Gráfico comparativo de preços normalizados
    plt.figure(figsize=(14, 7))
    
    for ticker, dados_acao in dados_multiplos.items():
        hist = dados_acao['historico']
        if hist is not None and not hist.empty:
            # Normalizar para base 100
            precos_norm = (hist['Close'] / hist['Close'].iloc[0]) * 100
            plt.plot(hist.index, precos_norm, linewidth=2, label=ticker)
    
    plt.title('Comparativo de Performance (Base 100)', fontsize=14, fontweight='bold')
    plt.xlabel('Data', fontsize=12)
    plt.ylabel('Índice (Base 100)', fontsize=12)
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 12. Área de Experimentação Livre

Use esta seção para seus próprios experimentos e análises.

In [ ]:
# Seu código aqui


---

## 📚 Recursos Adicionais

### Documentação útil:
- **BraPI:** https://brapi.dev/docs
- **Pandas:** https://pandas.pydata.org/docs/
- **Matplotlib:** https://matplotlib.org/
- **Seaborn:** https://seaborn.pydata.org/
- **Scikit-learn:** https://scikit-learn.org/

### Exemplos de análises:
1. Backtesting de estratégias
2. Otimização de hiperparâmetros dos modelos
3. Feature engineering avançado
4. Análise de sentimento com notícias
5. Comparação com índices (IBOV, etc)

---